# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 50
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus

In [3]:
import distributionAlgorithmPolling
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecast
import distributionAlgorithmForecastPolling

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()

# Auswahl des Algorithmus
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_FIFO
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_EVEN_DISTRIBUTION
# distributionAlgorithmPolling.start_simulation(solarpeakleistung, maximale_ladezeit, ladeleistung_pro_bev, simulation_day, bev_data, table_dict, simulation_data, algorithm, minute_interval)

#distributionAlgorithmForecast.start_algorithm(simulation_data, simulation_day, maximale_ladezeit, solarpeakleistung, 
  #                  bev_data, table_dict, ladeleistung_pro_bev, minute_interval)

distributionAlgorithmForecastPolling.start_simulation(solarpeakleistung, ladeleistung_pro_bev,
                     simulation_day, bev_data, table_dict, simulation_data, minute_interval)



Minute:  480
Charging BEVs after updating parking end:  []
Waiting BEVs:  [1, 9, 10, 11, 21, 25, 28, 32, 42]
New calculated number of charging stations:  1
Charging BEVs list before saving:  [1]
Number of charging BEVs 1
Charging BEV Liste vor Residual time Berechnung:  [1]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 1:  4.1775
Charging BEVs:  [1]


Minute:  495
Charging BEVs after updating parking end:  [1]
Waiting BEVs:  [9, 10, 11, 21, 25, 28, 32, 42]
New calculated number of charging stations:  1
Charging BEVs list before saving:  [1]
Number of charging BEVs 1
Charging BEV Liste vor Residual time Berechnung:  [1]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 1:  4.0225
Charging BEVs:  [1]


Minute:  510
Charging BEVs after updating parking end:  [1]
Waiting BEVs:  [9, 10, 11, 21, 25, 28, 32, 42, 0, 8, 12, 17, 19, 20, 24, 30, 31, 34, 39, 40, 41, 46, 48]
New calculated number of charging stations:  2
Charging BEVs list before saving:  [1, 9]
Number of char

Die Restladezeit des BEVS 42 endet vor dem nächsten Interval
Restladezeit:  10.046324818174341
Number of charging BEVs 5
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  12
Ladestart für ausgewähltes BEV:  640.0463248181743
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 0:  6.561964590964533
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 8:  5.010000000000007
Charging BEVs:  [21, 28, 42, 0, 8]


Minute:  645
Charging BEVs after updating parking end:  [21, 28, 0, 8, 12]
Waiting BEVs:  [17, 19, 20, 24, 30, 31, 34, 39, 40, 41, 46, 48, 13, 14, 36, 37, 43, 44, 15, 29, 38, 47]
New calculated number of charging stations:  5
Charging BEVs list before saving:  [21, 28, 0, 8, 12]
Number of charging BEVs 5
Number of charging BEVs 5
Number of charging BEVs 5
Number of charging BEVs 5
Number of charging BEVs 5
Charging BEV Liste vor Residual time Berechnung:  [21, 28, 0, 8, 12]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 21:  2.5850704838709238
Restladee

Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 19:  4.317386721133798
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 20:  5.006055391766727
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 24:  8.053761770712514
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 30:  7.89288620833002
Charging BEVs:  [28, 19, 20, 24, 30]


Minute:  765
Charging BEVs after updating parking end:  [28, 19, 20, 24, 30]
Waiting BEVs:  [34, 46, 48, 13, 14, 36, 37, 43, 15, 29, 38, 5, 7, 33, 6, 18, 22, 23, 27, 49]
New calculated number of charging stations:  4
Charging BEVs list before saving:  [28, 19, 20, 24, 30]
Number of charging BEVs 5
Number of charging BEVs 5
Number of charging BEVs 5
Number of charging BEVs 5
Number of charging BEVs 5
Charging BEV Liste vor Residual time Berechnung:  [28, 19, 20, 24, 30]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 28:  2.9407553846153958
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 19:  3.8795745324551807


Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 13:  6.973225310289525
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 14:  7.862500000000004
Charging BEVs:  [24, 30, 13, 14]


Minute:  900
Charging BEVs after updating parking end:  [24, 30, 13, 14]
Waiting BEVs:  [38, 5, 7, 33, 6, 18, 22, 23, 27, 49, 2, 3, 4, 16, 26, 35, 45]
New calculated number of charging stations:  2
Charging BEVs list before saving:  [24, 30, 13, 14]
Number of charging BEVs 4
Number of charging BEVs 4
Number of charging BEVs 4
Number of charging BEVs 4
Charging BEV Liste vor Residual time Berechnung:  [24, 30, 13, 14]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 24:  7.085057155300568
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 30:  6.034587455779528
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 13:  6.687015527623666
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 14:  7.504722222222231
Die Restparkzeit von BEV  30 endet vor dem nächsten Interv

### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag)

>**Ergebnis**<br>Geladene BEVs: 24 von 50<br>Aufgeladene Solarenergie insgesamt: 114.22 kWh<br>Ungenutzte Solarenergie insgesamt: 0 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [5]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480, step=15), Output()), _dom…

In [6]:
import figureGeneration

figureGeneration.create_charging_power_figure(simulation_day, solarpeakleistung, bev_data, minute_interval)

In [7]:
figureGeneration.create_bev_number_figure(simulation_data)

In [8]:
import postOptimization

postOptimization.sort_waiting_list_descending_by_parking_end(simulation_day)

{0: [(8.5, 5.6), 'wartend', [(626.3668387453449, 122.1944783494547, 7.175)], [7.175]], 1: [(8.0, 3.7), 'wartend', [(480, 53.53479853479848, 4.1775)], [4.1775]], 2: [(13.0, 5.7), 'wartend', [], [2.87]], 3: [(13.0, 4.8), 'wartend', [], [2.87]], 4: [(13.0, 5.9), 'wartend', [], [2.87]], 5: [(12.0, 3.1), 'nicht parkend', [], [3.7]], 6: [(12.5, 3.7), 'wartend', [], [3.435]], 7: [(12.0, 3.9), 'ladend', [(948, 12, 1.2746666666666666)], [4.185]], 8: [(8.5, 5.2), 'wartend', [(621.0657842749902, 50.88821255702317, 6.5175)], [6.5175]], 9: [(8.0, 6.0), 'wartend', [(510, 54.67258064516125, 7.02)], [7.02]], 10: [(8.0, 6.6), 'wartend', [(533.5347985347985, 92.83204021054644, 7.8999999999999995)], [7.8999999999999995]], 11: [(8.0, 5.7), 'wartend', [(540, 43.52307692307693, 6.6975)], [6.6975]], 12: [(8.5, 5.1), 'wartend', [(640.0463248181743, 77.57807224748956, 6.5175)], [6.5175]], 13: [(9.0, 7.2), 'nicht parkend', [(851.7290310552473, 120.27096894475267, 3.1625520450401705)], [8.3225]], 14: [(9.0, 6.8)

KeyError: <simulationClasses.SimulationDay object at 0x000001B6FCC5CC10>